In [1]:
# !pip install transformers torch shap
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import shap

In [11]:
data = pd.read_csv('promise_nfr.csv',sep=';',usecols=["RequirementText","NFR"])

,RequirementText,NFR
0,The system shall refresh the display every 60 ...,1
1,The application shall match the color of the s...,1
2,If projected the data must be readable. On ...,1
3,The product shall be available during normal ...,1
4,If projected the data must be understandable...,1
...,...,...
620,\tUser access should be limited to the permiss...,1
621,\tThe product must comply with the intranet p...,1
622,The intranet pages should display appropriate...,1
623,\tThe users should be able to easily use the ...,1


In [41]:
# 分割数据集
train_text, val_text, train_labels, val_labels = train_test_split(data['RequirementText'], data['NFR'], test_size=0.2)
train_text = train_text.iloc[:]

pandas.core.series.Series

**DistilBert**

In [54]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased").cuda()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [55]:
# use the custom function
import scipy as sp
def f(x):
    tv = torch.tensor([tokenizer.encode(text, padding='max_length', max_length=128, truncation=True) for text in x]).cuda()
    # tv = torch.tensor([tokenizer.encode(text,
    #   padding=True,
    #   truncation=True,
    #   max_length=512,
    #   return_tensors="pt")])

    attention_mask = (tv!=0).type(torch.int64).cuda()
    outputs = model(tv,attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val

In [56]:
explainer = shap.Explainer(f,tokenizer,output_names=["FR","NFR"])

In [57]:
shap_values = explainer(train_text[:3])

In [59]:
shap.plots.text(shap_values)

**pipeline**